In [1]:
!pip install pysrt


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [26]:

from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os 
import pandas as pd 

In [27]:
load_dotenv()

True

In [28]:
import pandas as pd

# Read your files here 
files = ['NEET_video_subtitles.txt'] #  you can add file or bunch of files
data = {}
for f in files:
  with open (f, "r") as myfile:
    all_lines = myfile.read().splitlines() # split by line
    # assign every alternative line to Time and Text index alternatively
    data['Time'], data['Text'] = all_lines[::2],  all_lines[1::2]

# create dataframe from the dictionary
df = pd.DataFrame(data)
print(df)    
youtube_link = "https://www.youtube.com/watch?v=dHXTb3_0DcQ&t"

         Time                                               Text
0        0:06  welcome to this Mega revision of microbiology ...
1        0:14  definitely a very volatile one and must be giv...
2        0:19  obviously in this less time you have to focus ...
3        0:24  most high yielding topics which we are gonna c...
4        0:30  session and I'm going to cover the images and ...
...       ...                                                ...
1423  2:30:11  and have confidence that whatever you're study...
1424  2:30:17  with volatile topics like this you need to hav...
1425  2:30:22  and I hope that this session has acted as a co...
1426  2:30:29  listening so carefully and yes feedback is alw...
1427  2:30:35  subscribe and press the Bell icon so you never...

[1428 rows x 2 columns]


In [24]:
from datetime import datetime

def split_by_timewindow(df, window_in_minutes):
    docs = []
    t = ''
    split_time = datetime.time(hour=0,minute=window_in_minutes, second=0)
    for index, row in df.iterrows():
        print (row['Time'], row['Text'] )
        newTime = datetime.strptime(row['Time'], '%H:%M:%S')
        if newTime < split_time:            
            t = t + ' ' + row['Text']
        else:            
            start_time =  (split_time.hour*3600) + (split_time.minute*60) + (split_time.second) - 60
            docs.append(Document(page_content=t,metadata={"link":f"{youtube_link}&t={start_time}s", "start_time":start_time}))
            t = row['Text']            
            split_time = (datetime.datetime(1, 1, 1,split_time.hour,split_time.minute, split_time.second) +  datetime.timedelta(minutes=window_in_minutes)).time()                
    print (docs)
    return docs

In [25]:
t=split_by_timewindow(df, 1)

TypeError: unbound method datetime.time() needs an argument

In [ ]:

embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    embedding=embedding,
    collection_name="test",
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
)

In [ ]:
vstore.add_documents(t)